In [1]:
import os
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp/research'

In [2]:
os.chdir("../")
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    year: int
    month: int
    root_dir: Path
    data_path: Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.dataDetails

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            year=params.year,
            month=params.month,
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [6]:
import os
from mlProject import logger
import pandas as pd

In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def process_data(self):
        logger.info(f'reading file - {self.config.data_path}')
        df = pd.read_parquet(self.config.data_path)
        
        categorical = ['PULocationID', 'DOLocationID']
        df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
        df['duration'] = df.duration.dt.total_seconds() / 60

        df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

        df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
        df['ride_id'] = f'{self.config.year:04d}/{self.config.month:02d}_' + df.index.astype('str')

        df.to_csv(os.path.join(self.config.root_dir, "predSet.csv"),index = False)
        logger.info("Data transformation completed")
        logger.info(df.shape)
        
        return df

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.process_data()
except Exception as e:
    raise e

[2024-07-01 06:41:07,987: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-01 06:41:07,989: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-01 06:41:07,991: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-01 06:41:07,992: INFO: common: created directory at: artifacts]
[2024-07-01 06:41:07,993: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-01 06:41:07,993: INFO: 1204532097: reading file - artifacts/data_ingestion/yellow_tripdata_2023-03.parquet]
[2024-07-01 06:41:47,605: INFO: 1204532097: Data transformation completed]
[2024-07-01 06:41:47,608: INFO: 1204532097: (3316216, 21)]
